Since [OpenPaths](https://openpaths.cc/) is dead, we switched to [WHIB](http://www.bleatinc.com/), which isn't [owned by Facebook](https://moves-app.com/) and doesn't [kill the phones battery](http://www.bigpaua.com/arc/).
A 2$ `premium` feature let's you export all the data as CSV, which is easy to parse and display.

In [1]:
import pandas
import matplotlib.pylab as plt
import seaborn
import mplleaflet
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
plt.rcParams['figure.figsize'] = (16, 9)  # We live in a widescreen world

In [3]:
locations = pandas.read_csv('journey.csv')

In [4]:
locations.head()

,Crumb,LocalDate,LocalTime,Latitude,Longitude,Altitude (in metres),Accuracy (in metres)
0,0,03.02.18,23:37,46.935401,7.418162,553,65
1,1,04.02.18,09:01,46.935344,7.417897,553,50
2,2,04.02.18,09:02,46.932949,7.422317,553,57
3,3,04.02.18,09:12,46.932855,7.422384,565,38
4,4,04.02.18,09:14,46.923639,7.415233,565,61


In [5]:
locations.tail()

,Crumb,LocalDate,LocalTime,Latitude,Longitude,Altitude (in metres),Accuracy (in metres)
14388,14388,09.01.19,18:15,46.940836,7.418846,546,65
14389,14389,09.01.19,18:16,46.940993,7.418745,0,79
14390,14390,09.01.19,18:18,46.935581,7.417921,0,95
14391,14391,09.01.19,18:18,46.935573,7.418067,0,0
14392,14392,09.01.19,18:23,46.935289,7.418211,550,165


In [6]:
locations.describe()

,Crumb,Latitude,Longitude,Altitude (in metres),Accuracy (in metres)
count,14393.000000,14393.000000,14393.000000,14393.000000,14393.000000
mean,7196.000000,46.551399,7.647089,534.772181,210.073369
std,4155.045547,1.655602,0.565627,421.566215,2261.191264
min,0.000000,39.118591,6.630272,-143.000000,0.000000
25%,3598.000000,46.932575,7.418067,492.000000,43.000000
50%,7196.000000,46.938937,7.428877,547.000000,65.000000
75%,10794.000000,46.948962,7.451904,556.000000,82.000000
max,14392.000000,49.479696,9.788838,2724.000000,149000.000000


In [7]:
tileprovider = 'Cartodb Positron'
# tileprovider = 'Stamen Toner'

In [8]:
zoom_start = 5

In [9]:
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
               tiles=tileprovider,
               zoom_start=zoom_start)
single = False
fast = True
if single:
    for c, loc in locations.iterrows():
        # not everyone, but every fifth one
        if not c % 5:
            folium.Marker(location=[loc.Latitude, loc.Longitude]).add_to(m)
else:
    if fast:
        # FastMarkerCluster
        m.add_child(FastMarkerCluster(locations[['Latitude', 'Longitude']].values.tolist()))
    else:
        # Markercluster
        mc = FastMarkerCluster()
        for c, loc in locations.iterrows():
            mc.add_child(folium.Marker(location=[loc.Latitude, loc.Longitude]))
        m.add_child(mc)
m

In [10]:
# from here: https://www.thedataschool.co.uk/gwilym-lockwood/viridis-colours-tableau/
gradient={0: '#440154FF',
          0.05: '#481567FF', 
          0.1: '#482677FF', 
          0.15: '#453781FF', 
          0.2: '#404788FF', 
          0.25: '#39568CFF', 
          0.3: '#33638DFF', 
          0.35: '#2D708EFF', 
          0.4: '#287D8EFF', 
          0.45: '#238A8DFF', 
          0.5: '#1F968BFF', 
          0.55: '#20A387FF', 
          0.6: '#29AF7FFF', 
          0.65: '#3CBB75FF', 
          0.7: '#55C667FF', 
          0.75: '#73D055FF', 
          0.8: '#95D840FF', 
          0.85: '#B8DE29FF', 
          0.9: '#DCE319FF', 
          0.95: '#FDE725FF'}

In [11]:
# Massage data to get a heatmap
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
               tiles=tileprovider,
               zoom_start=zoom_start)
folium.plugins.HeatMap(locations[['Latitude', 'Longitude']].values.tolist(),
                       gradient=gradient).add_to(m)
m